In [8]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
writer = SummaryWriter()

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [10]:
df = pd.read_csv("/home/vaclavmatejka/devel/janestreet/jane-street-market-prediction/encoded.csv", nrows=10000)
#df.loc[400:,"enc_feature_0":"enc_feature_49"]
#strip = 1 + df.shape[0] % 200
#df.loc[:df.shape[0]-strip]
#df["enc_feature_0", "feature_0"]
df.insert(3, "trade", None)
df.loc[df["resp"] <= 0, "trade"] = -1
df.loc[df["resp"] > 0, "trade"] = 1
df

,Unnamed: 0,date,weight,trade,resp_1,resp_2,resp_3,resp_4,resp,feature_0,...,enc_feature_40,enc_feature_41,enc_feature_42,enc_feature_43,enc_feature_44,enc_feature_45,enc_feature_46,enc_feature_47,enc_feature_48,enc_feature_49
0,0,0.0,0.000000,1,0.009916,0.014079,0.008773,0.001390,0.006270,1.0,...,-1.270642,0.472870,-0.425598,-2.030584,-0.095475,1.022529,-0.844325,0.304729,-2.480164,1.809270
1,1,0.0,16.673515,-1,-0.002828,-0.003226,-0.007319,-0.011114,-0.009792,-1.0,...,-0.538386,-1.621020,0.834621,-0.045175,-0.741891,0.184812,1.613565,-1.844530,-1.824630,2.063992
2,2,0.0,0.000000,1,0.025134,0.027607,0.033406,0.034380,0.023970,-1.0,...,-1.714202,0.512180,0.477450,-1.506564,-1.126838,0.460320,1.060316,0.283777,-3.106407,2.740955
3,3,0.0,0.000000,-1,-0.004730,-0.003273,-0.000461,-0.000476,-0.003200,-1.0,...,-0.916272,-1.159826,1.123091,-0.861372,-1.256033,1.031510,1.973300,-0.071008,-0.818719,2.005070
4,4,0.0,0.138531,-1,0.001252,0.002165,-0.001215,-0.006219,-0.002604,1.0,...,0.415481,0.663740,-1.612157,-2.534616,0.916429,-0.423991,0.352414,-0.623563,-2.533278,2.011271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,1.0,0.420165,1,0.001847,0.016171,0.039813,0.061120,0.051909,-1.0,...,2.232531,-0.919349,1.614605,0.220732,0.781079,-1.762073,-2.258388,0.836331,0.595957,-1.004861
9996,9996,1.0,2.852259,-1,-0.000712,-0.004864,-0.009795,-0.018601,-0.019537,1.0,...,2.564173,-0.511025,-0.835089,0.013764,-1.044754,-0.040393,-1.682571,-2.390744,0.361608,2.615752
9997,9997,1.0,1.389255,1,0.002383,0.004190,0.002356,0.006833,0.009608,1.0,...,-0.237486,-0.085354,-1.651367,-0.670482,-0.513578,-0.488636,-1.267944,-1.284699,0.015640,0.364390
9998,9998,1.0,0.085839,-1,-0.003532,-0.037491,-0.053815,-0.052844,-0.065972,1.0,...,0.508833,-0.126617,-2.936359,-2.490077,-2.401220,2.808005,-3.097896,-0.867709,0.529481,2.361072


In [4]:
t = torch.tensor([1.0,2000.0,3.0])
noise = torch.randn_like(t)
noise.multiply(1)

tensor([-0.4874, -1.1828, -0.3263])

In [5]:
VALIDATION_SIZE = 1000
p=0.1
df = pd.read_csv("/home/vaclavmatejka/devel/janestreet/jane-street-market-prediction/train.csv", nrows=10000, skiprows=lambda i: i > 0 and np.random.random() > p)


In [6]:
df = df.fillna(0)
df.loc[df['resp'] <= 0, 'trade'] = 0
df.loc[df['resp'] > 0, 'trade'] = 1

In [7]:
feature_0_pos = df.columns.get_loc("feature_0")
df.insert(feature_0_pos, "w_date", None)
df.insert(feature_0_pos, "w_ts_id", None)
df.loc[:, "w_date"] = (df["date"] / df["date"].max()) * 10
for one in range(df["date"].max()+1):
    min_id = df.loc[df["date"] == one, :]["ts_id"].min()
    max_id = df.loc[df["date"] == one, :]["ts_id"].max()
    distance = max_id - min_id
    df.loc[df["date"] == one, "w_ts_id"] = (df.loc[df["date"] == one, :]["ts_id"] - min_id) / distance

df.iloc[4000]["w_ts_id"]

0.7366515837104073

In [8]:
class JaneStreetDataset(Dataset):

    # Constructor with defult values
    def __init__(self, df, transform=None):
        self.df = df
        self.len = len(self.df)
        self.transform = transform

    # Getter
    def __getitem__(self, index):
        sample = torch.from_numpy(self.df.iloc[index]["feature_0":"feature_129"].values), torch.tensor(self.df.iloc[index]["resp"])
        sample = sample[0].to(device), sample[1].to(device)
        if self.transform:
            sample = self.transform(sample)
        return sample

    # Get Length
    def __len__(self):
        return self.len

In [9]:
df.iloc[2]["resp"]

0.010657450355787305

In [10]:
df.iloc[2]["feature_0":"feature_129"].values

array([1, 0.4309994855102589, -0.56656447366909, 0.9173147504036292,
       0.4740209628811886, 0.8680641224473489, 0.4848058293948703, 0.0,
       0.0, 0.9304045758114388, -0.12719137318286966, 0.0, 0.0,
       2.1797353711169336, 0.9885256898461467, 1.1614696571391283,
       0.3814523209688711, 0.0, 0.0, 2.054515587190643,
       1.5186244781980016, 0.0, 0.0, 3.01550530462673, 2.591359696948621,
       2.3744079683277852, 1.691290967949599, 0.0, 0.0,
       -0.1112465826145948, -0.1255963088203609, 0.0, 0.0,
       -0.015873817169229976, -0.014315091291192081, -0.08335632835247217,
       -0.08377188780165852, 1.157410408097431, 0.7625436514408253,
       0.009306697715007223, 0.018458686324766358, 0.5370037693713132,
       -0.9167707744600301, 2.261787963568694, 0.7483851622022146,
       1.8319323046102087, -0.549480978941606, -0.8081752907322435,
       -1.0176971520189724, -1.0484148632280508, -0.43194249486048575,
       -0.8297540630649565, -1.6932307001919789, 0.195233569208

In [11]:
DROPOUT_P = 0.0
big_number = 1000
small_number = 500
model = torch.nn.Sequential(
    torch.nn.Linear(130, small_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(small_number),
    torch.nn.Linear(small_number, big_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(big_number),
    torch.nn.Linear(big_number, big_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(big_number),
    torch.nn.Linear(big_number, big_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(big_number),
    torch.nn.Linear(big_number, small_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(small_number),
    torch.nn.Linear(small_number, 1),
)
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

model.apply(init_weights)


<ipython-input-11-ae5219358239>:29: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)


Sequential(
  (0): Linear(in_features=130, out_features=500, bias=True)
  (1): Dropout(p=0.0, inplace=False)
  (2): ReLU()
  (3): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): Linear(in_features=500, out_features=1000, bias=True)
  (5): Dropout(p=0.0, inplace=False)
  (6): ReLU()
  (7): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): Linear(in_features=1000, out_features=1000, bias=True)
  (9): Dropout(p=0.0, inplace=False)
  (10): ReLU()
  (11): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): Linear(in_features=1000, out_features=1000, bias=True)
  (13): Dropout(p=0.0, inplace=False)
  (14): ReLU()
  (15): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (16): Linear(in_features=1000, out_features=500, bias=True)
  (17): Dropout(p=0.0, inplace=False)
  (18): ReLU()
  (19): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, 

In [12]:
janestreet_train = JaneStreetDataset(df=df.iloc[:-VALIDATION_SIZE])
janestreet_validation = JaneStreetDataset(df=df.iloc[-VALIDATION_SIZE:])
BATCH_SIZE = 100
N_EPOCHS = 50
LR = 0.05
train_loader = torch.utils.data.DataLoader(dataset=janestreet_train, batch_size=BATCH_SIZE, drop_last=True)
validation_loader = torch.utils.data.DataLoader(dataset=janestreet_validation, batch_size=BATCH_SIZE, drop_last=True)

In [13]:
model = model.float()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.001)
criterion = nn.MSELoss()

In [14]:
def train_model(model, train_loader, validation_loader, optimizer, criterion, n_epochs=None):
    accuracy_list = []
    loss_list = []
    for epoch in range(n_epochs):
        for x, y in train_loader:
            model.train()
            optimizer.zero_grad()
            z = model(x.float())
            y = y.view(-1, 1).float()
            loss = criterion(z, y)
            writer.add_scalar("Train data", loss, epoch)
            loss.backward()
            optimizer.step()
            loss_list.append(loss.data)
        # perform a prediction on the validation data
        for x_test, y_test in validation_loader:
            model.eval()
            z = model(x_test.float())
            y = y.view(-1, 1)
            loss = criterion(z, y.float())
            writer.add_scalar("Validation data", loss, epoch)
            accuracy_list.append(loss.data)
    return accuracy_list, loss_list

In [15]:
accuracy_list, loss_list = train_model(model, train_loader, validation_loader, optimizer, criterion, n_epochs=N_EPOCHS)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [ ]:
model.eval()

In [ ]:
index=2000
#a = torch.from_numpy(df.iloc[index]["feature_0":"feature_129"].values)
a = df.iloc[index]["feature_0":"feature_129"].values
resp = torch.tensor(df.iloc[index]["resp"])

r = []
for one in range(BATCH_SIZE):
    r.append(a)
r = np.array(r)
model_input =torch.from_numpy(r).float().to(device)
z = model(model_input)

(resp, z[0])

In [ ]:
accuracy_list

In [ ]:
loss_list